# Using dask-distributed on keeling

## Packages to install via `pip` or `conda`
- `dask`
- `dask-distributed`
- `ipywidgets`

## Other steps
Copy `/data/keeling/a/snesbitt/.config/dask/jobqueue.yaml` to your `$HOME/.config/dask/jobqueue.yaml`.

## Notes
Due to the configuration of keeling, you can either run on g or h nodes only.  This setup will only work on g nodes.  If you want to work on h nodes, then edit `$HOME/.config/dask/jobqueue.yaml` and change `g20` to `h20` under `job-extra:`. Don't try to use both, you will have issues with the network configuration.

## Getting started
Start up your notebook on the keeling head node.  You may want to use the `screen` command to have a semi-permanent session running on there.  You can use `screen -r` to re-enter the session if you get disconnected.

Start up a jupyter notebook session as normal on that session.

`jupyter notebook --port=XXXX --ip=127.0.0.1 --no-browser`

Then, ssh to keeling using that port:

`ssh keeling.earth.illinois.edu -L XXXX:127.0.0.1:XXXX`

In [ ]:
from dask_jobqueue import SLURMCluster

The configuration in `jobqueue.yaml` will use 20 cores on each `keeling` node by default.

In [3]:
cluster = SLURMCluster()

/data/keeling/a/snesbitt/anaconda3/envs/py3k/lib/python3.7/site-packages/distributed/deploy/local.py:138: UserWarning: diagnostics_port has been deprecated. Please use `dashboard_address=` instead
  "diagnostics_port has been deprecated. "


Let's scale the cluster up to 4 nodes and 80 cores, shall we?

In [4]:
cluster.scale(4)

In [5]:
from dask.distributed import Client

In [6]:
client = Client(cluster)

In [7]:
client

Client Scheduler: tcp://172.22.178.4:33799,Cluster Workers: 4 Cores: 80 Memory: 400.00 GB


Here is where you define the function for computation to map to the cluster.

In [9]:
import time
def slow_increment(x):
    time.sleep(1)
    return x + 1

And here, let's map the jobs to the cluster.  This could be a file list or a range of numbers as here.

In [10]:
from dask.distributed import progress

In [11]:
futures = client.map(slow_increment,range(10000))

In [12]:
progress(futures)

VBox()

In [13]:
client.close()

In [14]:
cluster.close()